### Extras

In [17]:
import sys
import json
import pandas as pd

sys.path.append("..")

In [18]:
df = pd.read_excel("../data/colors and feelings.xlsx")
records = df.to_dict(orient="records")
records[:3]

[{'name': 'red_light ',
  'emotion1': ' energetic ',
  'emotion2': ' warm ',
  'hex': ' #F4B7B7'},
 {'name': 'red_mid ',
  'emotion1': ' passionate ',
  'emotion2': ' intense ',
  'hex': ' #D96B6B'},
 {'name': 'red_dark ',
  'emotion1': ' bold ',
  'emotion2': ' fierce ',
  'hex': ' #B33232'}]

In [19]:
with open("../data/colors and feelings.json", "w") as f:
    json.dump(records, f, indent=4)


In [20]:
with open("../data/colors and feelings.json") as f:
    colors = json.load(f)

colors[:3]

[{'name': 'red_light ',
  'emotion1': ' energetic ',
  'emotion2': ' warm ',
  'hex': ' #F4B7B7'},
 {'name': 'red_mid ',
  'emotion1': ' passionate ',
  'emotion2': ' intense ',
  'hex': ' #D96B6B'},
 {'name': 'red_dark ',
  'emotion1': ' bold ',
  'emotion2': ' fierce ',
  'hex': ' #B33232'}]

In [21]:
from app.utils.color_utils import hex_to_lab

clean_colors = []

for c in colors:
    if not c["name"] or not c["hex"]:
        continue
    
    name = str(c["name"]).strip()
    e1 = str(c["emotion1"]).strip()
    e2 = str(c["emotion2"]).strip()
    hex_code = str(c["hex"]).strip()


    if not hex_code.startswith("#"):
        continue

    lab = hex_to_lab(hex_code)
    emotion = f"{e1} {e2}"

    clean_colors.append({
        "name": name,
        "hex": hex_code,
        "emotion": emotion,
        "lab": lab
    })

clean_colors[:3]


[{'name': 'red_light',
  'hex': '#F4B7B7',
  'emotion': 'energetic warm',
  'lab': (79.90962552269028, 22.058583944627564, 8.631237276151937)},
 {'name': 'red_mid',
  'hex': '#D96B6B',
  'emotion': 'passionate intense',
  'lab': (58.350564898855666, 43.04572491142683, 20.352869492562277)},
 {'name': 'red_dark',
  'hex': '#B33232',
  'emotion': 'bold fierce',
  'lab': (41.37073203682336, 51.624549599757344, 30.895689315536977)}]

### Day 4

In [22]:
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color

def hex_to_rgb(hex_code):
    hex_code = hex_code.lstrip("#")
    return tuple(int(hex_code[i:i+2], 16) for i in (0, 2, 4))

def hex_to_lab(hex_code):
    hex_code = hex_code.lstrip("#")
    r = int(hex_code[0:2], 16)
    g = int(hex_code[2:4], 16)
    b = int(hex_code[4:6], 16)
    
    rgb = sRGBColor(r, g, b, is_upscaled=True)
    lab = convert_color(rgb, LabColor)
    
    return (lab.lab_l, lab.lab_a, lab.lab_b)

colors = ["#4A6C89", "#3498DB", "#2ECC71"]

for c in colors:
    rgb = hex_to_rgb(c)
    lab = hex_to_lab(c)

print(f"Color: {c}")
print(f"RGB: {rgb}")
print(f"LAB: {lab}")
print()

Color: #2ECC71
RGB: (46, 204, 113)
LAB: (72.86304893124685, -59.7152758953215, 34.19613800009999)



### Day 9

In [23]:
from app.utils.emotion_utils import get_vad_for_emotion

combined_colors = []

for c in clean_colors:
    lab = c["lab"]
    v, a, d = get_vad_for_emotion(c["emotion"])
    vad = [v, a, d]
    vector = list(lab) + vad
    
    combined_colors.append({
        "name": c["name"],
        "hex": c["hex"],
        "lab": lab,
        "emotion": c["emotion"],
        "vad": vad,
        "vector": vector
    })

combined_colors[:2]


[{'name': 'red_light',
  'hex': '#F4B7B7',
  'lab': (79.90962552269028, 22.058583944627564, 8.631237276151937),
  'emotion': 'energetic warm',
  'vad': [0.694, 0.736, 0.804],
  'vector': [79.90962552269028,
   22.058583944627564,
   8.631237276151937,
   0.694,
   0.736,
   0.804]},
 {'name': 'red_mid',
  'hex': '#D96B6B',
  'lab': (58.350564898855666, 43.04572491142683, 20.352869492562277),
  'emotion': 'passionate intense',
  'vad': [0.99, 0.448, 0.498],
  'vector': [58.350564898855666,
   43.04572491142683,
   20.352869492562277,
   0.99,
   0.448,
   0.498]}]

### Day 10

In [24]:
for item in combined_colors:
    print("Name:", item["name"])
    print("Lab:", item["lab"])
    print("VAD:", item["vad"])
    print("Vector:", item["vector"])
    print("-" * 40)

Name: red_light
Lab: (79.90962552269028, 22.058583944627564, 8.631237276151937)
VAD: [0.694, 0.736, 0.804]
Vector: [79.90962552269028, 22.058583944627564, 8.631237276151937, 0.694, 0.736, 0.804]
----------------------------------------
Name: red_mid
Lab: (58.350564898855666, 43.04572491142683, 20.352869492562277)
VAD: [0.99, 0.448, 0.498]
Vector: [58.350564898855666, 43.04572491142683, 20.352869492562277, 0.99, 0.448, 0.498]
----------------------------------------
Name: red_dark
Lab: (41.37073203682336, 51.624549599757344, 30.895689315536977)
VAD: [-0.438, 0.754, 0.518]
Vector: [41.37073203682336, 51.624549599757344, 30.895689315536977, -0.438, 0.754, 0.518]
----------------------------------------
Name: peach_light
Lab: (82.74524732069881, 15.723937614188, 12.666969846897214)
VAD: [0.898, -0.156, 0.1]
Vector: [82.74524732069881, 15.723937614188, 12.666969846897214, 0.898, -0.156, 0.1]
----------------------------------------
Name: peach_mid
Lab: (63.16167640606969, 31.1312705837557, 